# Data Storage in Database

### Import Libraries

In [1]:
# Libraries
import os
import mysql.connector
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Funktion, um eine MySQL-Verbindung zu prüfen
def check_conn(conn):
    try:
        conn.cursor()
        return True
    except Exception as ex:
        return False

# Aktuelles Arbeitsverzeichnis abrufen
print("Aktuelles Verzeichnis:", os.getcwd())


Aktuelles Verzeichnis: c:\DataAnalytics\DataAnalyticsLN\Data Storage in Database


# Load the datasets

In [5]:
unhcr_file_path = 'unhcr_refugee_detailed_data.csv'
gdp_file_path = 'refugee_data_and_gdp.csv'

unhcr_data = pd.read_csv(unhcr_file_path)
gdp_data = pd.read_csv(gdp_file_path)

# Display the first few rows of both datasets to understand their structure
print("UNHCR Data:")
unhcr_data.head()



UNHCR Data:


,year,coo_id,coa_id,coo_name,coa_name,coo,coa,coo_iso,coa_iso,f_0_4,...,f_other,f_total,m_0_4,m_5_11,m_12_17,m_18_59,m_60,m_other,m_total,total
0,2001,2,202,Afghanistan,United States of America,AFG,USA,AFG,USA,0,...,0,0,0,0,0,0,0,0,0,7756
1,2002,2,202,Afghanistan,United States of America,AFG,USA,AFG,USA,0,...,0,0,0,0,0,0,0,0,0,9227
2,2003,2,202,Afghanistan,United States of America,AFG,USA,AFG,USA,0,...,0,0,0,0,0,0,0,0,0,9821
3,2004,2,202,Afghanistan,United States of America,AFG,USA,AFG,USA,0,...,0,0,0,0,0,0,0,0,0,9923
4,2005,2,202,Afghanistan,United States of America,AFG,USA,AFG,USA,0,...,0,0,0,0,0,0,0,0,0,8931


In [6]:
print("Refugee and GDP Data:")
gdp_data.head()

Refugee and GDP Data:


,year,country_name,gdp,total_refugees,coo_name,total,male_female_ratio
0,2001,France,1.370377e+12,6036.0,Afghanistan,1284.0,1.221453
1,2001,France,1.370377e+12,6036.0,Syrian Arab Rep.,192.0,2.918367
2,2001,France,1.370377e+12,6036.0,Iran (Islamic Rep. of),1987.0,1.806497
3,2001,France,1.370377e+12,6036.0,Iraq,1629.0,1.291139
4,2001,France,1.370377e+12,6036.0,Ukraine,407.0,1.014851


In [8]:
## Data Preparation
# Clean and prepare the UNHCR data
print("Cleaning UNHCR Data...")
unhcr_data.dropna(inplace=True)  # Remove missing values
unhcr_data.drop_duplicates(inplace=True)  # Remove duplicates

# Clean and prepare the GDP data
print("Cleaning Refugee and GDP Data...")
gdp_data.dropna(inplace=True)
gdp_data.drop_duplicates(inplace=True)



Cleaning UNHCR Data...
Cleaning Refugee and GDP Data...


## MySQL-Connection

In [9]:
# MySQL-Verbindung herstellen
conn = mysql.connector.connect(
    host="localhost",  # Nur den Hostnamen angeben
    port=3306,         # Port separat definieren
    user="root",
    password="Leylaxanampuc1.",
    database="refugees"
)
cursor = conn.cursor()

# Verbindung prüfen
if check_conn(conn):
    print("Verbindung erfolgreich hergestellt!")
else:
    print("Verbindung fehlgeschlagen.")


Verbindung erfolgreich hergestellt!


In [10]:
# Create a new database
cursor.execute("CREATE DATABASE IF NOT EXISTS RefugeeAnalysis;")
conn.database = "RefugeeAnalysis"

In [11]:
# Create tables for the datasets
cursor.execute("""
CREATE TABLE IF NOT EXISTS UNHCR_Data (
    year INT,
    coo_id INT,
    coa_id INT,
    coo_name VARCHAR(255),
    coa_name VARCHAR(255),
    coo_iso VARCHAR(10),
    coa_iso VARCHAR(10),
    f_0_4 INT,
    f_5_11 INT,
    f_12_17 INT,
    f_18_59 INT,
    f_60 INT,
    f_other INT,
    f_total INT,
    m_0_4 INT,
    m_5_11 INT,
    m_12_17 INT,
    m_18_59 INT,
    m_60 INT,
    m_other INT,
    m_total INT,
    total INT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Refugee_GDP_Data (
    year INT,
    country_name VARCHAR(255),
    gdp FLOAT,
    total_refugees INT,
    coo_name VARCHAR(255),
    total INT,
    male_female_ratio FLOAT
);
""")


In [12]:
# Insert data into MySQL tables
print("Inserting data into MySQL...")
for _, row in unhcr_data.iterrows():
    cursor.execute("""
    INSERT INTO UNHCR_Data (year, coo_id, coa_id, coo_name, coa_name, coo_iso, coa_iso, f_0_4, f_5_11, f_12_17, f_18_59, f_60, f_other, f_total, m_0_4, m_5_11, m_12_17, m_18_59, m_60, m_other, m_total, total)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, tuple(row[["year", "coo_id", "coa_id", "coo_name", "coa_name", "coo_iso", "coa_iso", "f_0_4", "f_5_11", "f_12_17", "f_18_59", "f_60", "f_other", "f_total", "m_0_4", "m_5_11", "m_12_17", "m_18_59", "m_60", "m_other", "m_total", "total"]]))

for _, row in gdp_data.iterrows():
    cursor.execute("""
    INSERT INTO Refugee_GDP_Data (year, country_name, gdp, total_refugees, coo_name, total, male_female_ratio)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, tuple(row[["year", "country_name", "gdp", "total_refugees", "coo_name", "total", "male_female_ratio"]]))

conn.commit()
print("Data inserted successfully.")

Inserting data into MySQL...
Data inserted successfully.


## Close Connection

In [ ]:
# Verbindung schließen
cursor.close()
conn.close()
print("Verbindung geschlossen")